In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'


producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

# PRODUCE DATA

In [2]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'redpanda-1:9092'
server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {"number":i}
    producer.send(topic_name, value=message)
    print(f'Sent: {message}')
    time.sleep(0.05)
print(time.time()-t0)
producer.flush()
t1 = time.time()
print(f"Tool {(t1-t0)} senconds")

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
0.7978627681732178
Tool 0.7989053726196289 senconds


In [6]:
0.6194324493408203 - 0.6194324493408203 

0.0

In [12]:
import pandas as pd 

In [14]:
df_green = pd.read_csv('green_tripdata_2019-10.csv')

C:\Users\Windows\AppData\Local\Temp\ipykernel_20744\29539273.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_green = pd.read_csv('green_tripdata_2019-10.csv')


In [15]:
df_green_process = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'tip_amount']]

In [16]:
df_green_process

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26
...,...,...,...,...,...,...,...
476381,2019-10-31 23:30:00,2019-11-01 00:00:00,65,102,NaN,7.04,0.00
476382,2019-10-31 23:03:00,2019-10-31 23:24:00,129,136,NaN,0.00,0.00
476383,2019-10-31 23:02:00,2019-10-31 23:23:00,61,222,NaN,3.90,0.00
476384,2019-10-31 23:42:00,2019-10-31 23:56:00,76,39,NaN,3.08,0.00


In [17]:
topic_name = 'green-trips'
t0 = time.time()
for row in df_green_process.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic = topic_name, value=row_dict)
    
print(time.time()- t0)

49.73673176765442


# CONSUME DATA PYSPARK

In [5]:
import pyspark 
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"
# SparkSession.setLevel('INFO')
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Green Trips Consumer") \
    .config('spark.jars.packages', kafka_jar_package) \
    .getOrCreate()

In [6]:
spark

In [7]:
# sc = pyspark.SparkContext.getOrCreate()
# log4j = sc._jvm.org.apache.log4j
# log4j.LogManager.getRootLogger().setLevel(log4j.Level.DEBUG)

In [7]:
topic_name = 'green-trips'
server = 'redpanda-1:9092'
server = 'localhost:9092'
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", server) \
    .option("subscribe", topic_name) \
    .option('startingOffsets', 'earliest') \
    .load()

In [8]:
green_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [9]:
def  peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek)

In [10]:
query.start()

In [7]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [8]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")